# Manual Backpropagation

Okay, let's extend the explanation to a 3-layer MLP (L=3, meaning 2 hidden layers and 1 output layer) and add the derivative calculation with respect to the input `x`.

## Foundational Mathematical Explanation (L=3 MLP)

### 1. Network Architecture and Notation (L=3)

*   **Input Layer (Layer 0):** Input vector $\mathbf{x} \in \mathbb{R}^{n_0}$. Activation $\mathbf{a}^0 = \mathbf{x}$.
*   **Hidden Layer 1 (l=1):**
    *   Weight matrix $\mathbf{W}^1 \in \mathbb{R}^{n_1 \times n_0}$
    *   Bias vector $\mathbf{b}^1 \in \mathbb{R}^{n_1}$
    *   Pre-activation: $\mathbf{z}^1 = \mathbf{W}^1 \mathbf{a}^0 + \mathbf{b}^1$, $\mathbf{z}^1 \in \mathbb{R}^{n_1}$
    *   Activation (ReLU): $\mathbf{a}^1 = \text{ReLU}(\mathbf{z}^1) = \max(0, \mathbf{z}^1)$, $\mathbf{a}^1 \in \mathbb{R}^{n_1}$
*   **Hidden Layer 2 (l=2):**
    *   Weight matrix $\mathbf{W}^2 \in \mathbb{R}^{n_2 \times n_1}$
    *   Bias vector $\mathbf{b}^2 \in \mathbb{R}^{n_2}$
    *   Pre-activation: $\mathbf{z}^2 = \mathbf{W}^2 \mathbf{a}^1 + \mathbf{b}^2$, $\mathbf{z}^2 \in \mathbb{R}^{n_2}$
    *   Activation (ReLU): $\mathbf{a}^2 = \text{ReLU}(\mathbf{z}^2) = \max(0, \mathbf{z}^2)$, $\mathbf{a}^2 \in \mathbb{R}^{n_2}$
*   **Output Layer (l=3):**
    *   Weight matrix $\mathbf{W}^3 \in \mathbb{R}^{n_3 \times n_2}$
    *   Bias vector $\mathbf{b}^3 \in \mathbb{R}^{n_3}$
    *   Pre-activation: $\mathbf{z}^3 = \mathbf{W}^3 \mathbf{a}^2 + \mathbf{b}^3$, $\mathbf{z}^3 \in \mathbb{R}^{n_3}$
    *   Activation (Softmax): $\hat{\mathbf{y}} = \mathbf{a}^3 = \text{softmax}(\mathbf{z}^3)$, $\hat{\mathbf{y}} \in \mathbb{R}^{n_3}$. $n_3$ is the number of classes.
*   **Target:** True label vector $\mathbf{y} \in \mathbb{R}^{n_3}$ (one-hot).
*   **Loss Function (Categorical Cross-Entropy):**
    $J(\hat{\mathbf{y}}, \mathbf{y}) = - \sum_{i=1}^{n_3} y_i \log(\hat{y}_i)$

### 2. Goal: Gradient Calculation

Compute gradients w.r.t. all parameters ($\mathbf{W}^1, \mathbf{b}^1, \mathbf{W}^2, \mathbf{b}^2, \mathbf{W}^3, \mathbf{b}^3$) and the input $\mathbf{x}$.

### 3. Backpropagation Steps (L=3)

We use the chain rule, propagating the gradient of the loss `J` backward. The core quantities are the gradients w.r.t. pre-activations, $\delta^l = \frac{\partial J}{\partial \mathbf{z}^l}$.

**Step 1: Gradient w.r.t. Output Layer Pre-activation (z^3)**

As derived previously, for CCE loss and Softmax activation:
$\delta^3 = \frac{\partial J}{\partial \mathbf{z}^3} = \mathbf{a}^3 - \mathbf{y} = \hat{\mathbf{y}} - \mathbf{y}$

**Step 2: Gradients for Layer 3 Parameters (W^3, b^3)**

Using $\mathbf{z}^3 = \mathbf{W}^3 \mathbf{a}^2 + \mathbf{b}^3$:
*   $\frac{\partial J}{\partial \mathbf{W}^3} = \frac{\partial J}{\partial \mathbf{z}^3} \left(\frac{\partial \mathbf{z}^3}{\partial \mathbf{W}^3}\right)_{\text{tensor contraction}} = \delta^3 (\mathbf{a}^2)^T$
*   $\frac{\partial J}{\partial \mathbf{b}^3} = \frac{\partial J}{\partial \mathbf{z}^3} \frac{\partial \mathbf{z}^3}{\partial \mathbf{b}^3} = \delta^3 \mathbf{I} = \delta^3$

**Step 3: Propagate Error to Layer 2 Activation (a^2)**

$\frac{\partial J}{\partial \mathbf{a}^2} = \frac{\partial \mathbf{z}^3}{\partial \mathbf{a}^2}^T \frac{\partial J}{\partial \mathbf{z}^3} = (\mathbf{W}^3)^T \delta^3$

**Step 4: Propagate Error Through Layer 2 ReLU Activation (z^2)**

$\delta^2 = \frac{\partial J}{\partial \mathbf{z}^2} = \frac{\partial J}{\partial \mathbf{a}^2} \odot \frac{\partial \mathbf{a}^2}{\partial \mathbf{z}^2} = \frac{\partial J}{\partial \mathbf{a}^2} \odot \text{ReLU}'(\mathbf{z}^2)$
where $\odot$ is the Hadamard (element-wise) product and $\text{ReLU}'(z)$ is the element-wise derivative (1 if z > 0, 0 otherwise).
Substituting from Step 3:
$\delta^2 = [(\mathbf{W}^3)^T \delta^3] \odot \text{ReLU}'(\mathbf{z}^2)$

**Step 5: Gradients for Layer 2 Parameters (W^2, b^2)**

Using $\mathbf{z}^2 = \mathbf{W}^2 \mathbf{a}^1 + \mathbf{b}^2$:
*   $\frac{\partial J}{\partial \mathbf{W}^2} = \frac{\partial J}{\partial \mathbf{z}^2} \left(\frac{\partial \mathbf{z}^2}{\partial \mathbf{W}^2}\right)_{\text{tensor contraction}} = \delta^2 (\mathbf{a}^1)^T$
*   $\frac{\partial J}{\partial \mathbf{b}^2} = \frac{\partial J}{\partial \mathbf{z}^2} \frac{\partial \mathbf{z}^2}{\partial \mathbf{b}^2} = \delta^2 \mathbf{I} = \delta^2$

**Step 6: Propagate Error to Layer 1 Activation (a^1)**

$\frac{\partial J}{\partial \mathbf{a}^1} = \frac{\partial \mathbf{z}^2}{\partial \mathbf{a}^1}^T \frac{\partial J}{\partial \mathbf{z}^2} = (\mathbf{W}^2)^T \delta^2$

**Step 7: Propagate Error Through Layer 1 ReLU Activation (z^1)**

$\delta^1 = \frac{\partial J}{\partial \mathbf{z}^1} = \frac{\partial J}{\partial \mathbf{a}^1} \odot \frac{\partial \mathbf{a}^1}{\partial \mathbf{z}^1} = \frac{\partial J}{\partial \mathbf{a}^1} \odot \text{ReLU}'(\mathbf{z}^1)$
Substituting from Step 6:
$\delta^1 = [(\mathbf{W}^2)^T \delta^2] \odot \text{ReLU}'(\mathbf{z}^1)$

**Step 8: Gradients for Layer 1 Parameters (W^1, b^1)**

Using $\mathbf{z}^1 = \mathbf{W}^1 \mathbf{a}^0 + \mathbf{b}^1$:
*   $\frac{\partial J}{\partial \mathbf{W}^1} = \frac{\partial J}{\partial \mathbf{z}^1} \left(\frac{\partial \mathbf{z}^1}{\partial \mathbf{W}^1}\right)_{\text{tensor contraction}} = \delta^1 (\mathbf{a}^0)^T = \delta^1 \mathbf{x}^T$
*   $\frac{\partial J}{\partial \mathbf{b}^1} = \frac{\partial J}{\partial \mathbf{z}^1} \frac{\partial \mathbf{z}^1}{\partial \mathbf{b}^1} = \delta^1 \mathbf{I} = \delta^1$

**Step 9: Gradient w.r.t. Network Input (x = a^0)**

We need $\frac{\partial J}{\partial \mathbf{a}^0}$. This is computed similarly to Steps 3 and 6:
$\frac{\partial J}{\partial \mathbf{a}^0} = \frac{\partial \mathbf{z}^1}{\partial \mathbf{a}^0}^T \frac{\partial J}{\partial \mathbf{z}^1} = (\mathbf{W}^1)^T \delta^1$
Since $\mathbf{x} = \mathbf{a}^0$:
$\frac{\partial J}{\partial \mathbf{x}} = (\mathbf{W}^1)^T \delta^1$

**Full Chain Rule Formula for ∂J/∂x:**
Writing the full chain rule dependency explicitly:
$$
\frac{\partial J}{\partial \mathbf{x}} = \frac{\partial J}{\partial \mathbf{a}^3} \frac{\partial \mathbf{a}^3}{\partial \mathbf{z}^3} \frac{\partial \mathbf{z}^3}{\partial \mathbf{a}^2} \frac{\partial \mathbf{a}^2}{\partial \mathbf{z}^2} \frac{\partial \mathbf{z}^2}{\partial \mathbf{a}^1} \frac{\partial \mathbf{a}^1}{\partial \mathbf{z}^1} \frac{\partial \mathbf{z}^1}{\partial \mathbf{a}^0}
$$
Substituting the Jacobians (where $\mathbf{J}_{softmax}$, $\mathbf{J}_{ReLU}$ are Jacobian matrices):
$$
\frac{\partial J}{\partial \mathbf{x}}^T = \left(\frac{\partial J}{\partial \mathbf{a}^3}\right)^T \mathbf{J}_{softmax}(\mathbf{z}^3) (\mathbf{W}^3) \mathbf{J}_{ReLU}(\mathbf{z}^2) (\mathbf{W}^2) \mathbf{J}_{ReLU}(\mathbf{z}^1) (\mathbf{W}^1)
$$
Note: $\frac{\partial J}{\partial \mathbf{a}^3} \frac{\partial \mathbf{a}^3}{\partial \mathbf{z}^3} = (\delta^3)^T$. Also $\mathbf{J}_{ReLU}(\mathbf{z}^l)$ is a diagonal matrix with $\text{ReLU}'(z_i^l)$ on the diagonal. The chain rule involving element-wise operations and matrix multiplications can be more compactly written using the $\delta^l$ terms as derived above. The final result $\frac{\partial J}{\partial \mathbf{x}} = (\mathbf{W}^1)^T \delta^1$ correctly encapsulates this chain.

---

## Simple Numerical Example (3-Layer MLP)

Let's use a 3-layer MLP (2 hidden, 1 output).
*   Input $n_0 = 2$
*   Hidden Layer 1 $n_1 = 3$ (ReLU)
*   Hidden Layer 2 $n_2 = 4$ (ReLU)
*   Output Layer $n_3 = 2$ (Softmax, CCE Loss)

**Network Parameters:**
*   $\mathbf{W}^1 = \begin{pmatrix} 0.1 & 0.2 \\ 0.3 & 0.4 \\ 0.5 & 0.6 \end{pmatrix}$ (3x2), $\mathbf{b}^1 = \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \end{pmatrix}$ (3x1)
*   $\mathbf{W}^2 = \begin{pmatrix} 0.1 & 0.2 & 0.3 \\ 0.4 & 0.5 & 0.6 \\ 0.7 & 0.8 & 0.9 \\ 0.2 & 0.1 & 0.3 \end{pmatrix}$ (4x3), $\mathbf{b}^2 = \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \end{pmatrix}$ (4x1)
*   $\mathbf{W}^3 = \begin{pmatrix} 0.5 & 0.6 & 0.7 & 0.8 \\ 0.9 & 0.8 & 0.7 & 0.6 \end{pmatrix}$ (2x4), $\mathbf{b}^3 = \begin{pmatrix} 0.2 \\ 0.2 \end{pmatrix}$ (2x1)

**Input Sample:**
*   $\mathbf{x} = \mathbf{a}^0 = \begin{pmatrix} 1.0 \\ 0.5 \end{pmatrix}$

**Target Label:**
*   $\mathbf{y} = \begin{pmatrix} 0 \\ 1 \end{pmatrix}$ (representing class 1)

**Step-by-Step Calculation (Manual):**

**1. Forward Pass:**

*   **Layer 1:**
    *   $\mathbf{z}^1 = \mathbf{W}^1 \mathbf{a}^0 + \mathbf{b}^1 = \begin{pmatrix} 0.1 & 0.2 \\ 0.3 & 0.4 \\ 0.5 & 0.6 \end{pmatrix} \begin{pmatrix} 1.0 \\ 0.5 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \end{pmatrix} = \begin{pmatrix} 0.1+0.1 \\ 0.3+0.2 \\ 0.5+0.3 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \end{pmatrix} = \begin{pmatrix} 0.2 \\ 0.5 \\ 0.8 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \end{pmatrix} = \begin{pmatrix} 0.3 \\ 0.6 \\ 0.9 \end{pmatrix}$
    *   $\mathbf{a}^1 = \text{ReLU}(\mathbf{z}^1) = \begin{pmatrix} 0.3 \\ 0.6 \\ 0.9 \end{pmatrix}$

*   **Layer 2:**
    *   $\mathbf{z}^2 = \mathbf{W}^2 \mathbf{a}^1 + \mathbf{b}^2 = \begin{pmatrix} 0.1 & 0.2 & 0.3 \\ 0.4 & 0.5 & 0.6 \\ 0.7 & 0.8 & 0.9 \\ 0.2 & 0.1 & 0.3 \end{pmatrix} \begin{pmatrix} 0.3 \\ 0.6 \\ 0.9 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \end{pmatrix}$
        $= \begin{pmatrix} 0.03+0.12+0.27 \\ 0.12+0.30+0.54 \\ 0.21+0.48+0.81 \\ 0.06+0.06+0.27 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \end{pmatrix} = \begin{pmatrix} 0.42 \\ 0.96 \\ 1.50 \\ 0.39 \end{pmatrix} + \begin{pmatrix} 0.1 \\ 0.1 \\ 0.1 \\ 0.1 \end{pmatrix} = \begin{pmatrix} 0.52 \\ 1.06 \\ 1.60 \\ 0.49 \end{pmatrix}$
    *   $\mathbf{a}^2 = \text{ReLU}(\mathbf{z}^2) = \begin{pmatrix} 0.52 \\ 1.06 \\ 1.60 \\ 0.49 \end{pmatrix}$

*   **Layer 3 (Output):**
    *   $\mathbf{z}^3 = \mathbf{W}^3 \mathbf{a}^2 + \mathbf{b}^3 = \begin{pmatrix} 0.5 & 0.6 & 0.7 & 0.8 \\ 0.9 & 0.8 & 0.7 & 0.6 \end{pmatrix} \begin{pmatrix} 0.52 \\ 1.06 \\ 1.60 \\ 0.49 \end{pmatrix} + \begin{pmatrix} 0.2 \\ 0.2 \end{pmatrix}$
        $= \begin{pmatrix} 0.5(0.52)+0.6(1.06)+0.7(1.60)+0.8(0.49) \\ 0.9(0.52)+0.8(1.06)+0.7(1.60)+0.6(0.49) \end{pmatrix} + \begin{pmatrix} 0.2 \\ 0.2 \end{pmatrix}$
        $= \begin{pmatrix} 0.260+0.636+1.120+0.392 \\ 0.468+0.848+1.120+0.294 \end{pmatrix} + \begin{pmatrix} 0.2 \\ 0.2 \end{pmatrix} = \begin{pmatrix} 2.408 \\ 2.730 \end{pmatrix} + \begin{pmatrix} 0.2 \\ 0.2 \end{pmatrix} = \begin{pmatrix} 2.608 \\ 2.930 \end{pmatrix}$
    *   $\mathbf{a}^3 = \hat{\mathbf{y}} = \text{softmax}(\mathbf{z}^3)$
        $\exp(\mathbf{z}^3) \approx \begin{pmatrix} e^{2.608} \\ e^{2.930} \end{pmatrix} \approx \begin{pmatrix} 13.571 \\ 18.727 \end{pmatrix}$
        Sum $\approx 13.571 + 18.727 = 32.298$
        $\mathbf{a}^3 = \hat{\mathbf{y}} \approx \begin{pmatrix} 13.571 / 32.298 \\ 18.727 / 32.298 \end{pmatrix} \approx \begin{pmatrix} 0.4199 \\ 0.5801 \end{pmatrix}$

**2. Backward Pass:**

*   **Compute $\delta^3$ (Output Layer Error Signal):**
    $\delta^3 = \frac{\partial J}{\partial \mathbf{z}^3} = \mathbf{a}^3 - \mathbf{y} \approx \begin{pmatrix} 0.4199 \\ 0.5801 \end{pmatrix} - \begin{pmatrix} 0 \\ 1 \end{pmatrix} = \begin{pmatrix} 0.4199 \\ -0.4199 \end{pmatrix}$

*   **Compute Gradients for Layer 3 ($\mathbf{W}^3, \mathbf{b}^3$):**
    *   $\frac{\partial J}{\partial \mathbf{W}^3} = \delta^3 (\mathbf{a}^2)^T \approx \begin{pmatrix} 0.4199 \\ -0.4199 \end{pmatrix} \begin{pmatrix} 0.52 & 1.06 & 1.60 & 0.49 \end{pmatrix}$
        $\frac{\partial J}{\partial \mathbf{W}^3} \approx \begin{pmatrix} 0.4199 \cdot 0.52 & 0.4199 \cdot 1.06 & 0.4199 \cdot 1.60 & 0.4199 \cdot 0.49 \\ -0.4199 \cdot 0.52 & -0.4199 \cdot 1.06 & -0.4199 \cdot 1.60 & -0.4199 \cdot 0.49 \end{pmatrix}$
        $\frac{\partial J}{\partial \mathbf{W}^3} \approx \begin{pmatrix} 0.2183 & 0.4451 & 0.6718 & 0.2058 \\ -0.2183 & -0.4451 & -0.6718 & -0.2058 \end{pmatrix}$ (2x4)
    *   $\frac{\partial J}{\partial \mathbf{b}^3} = \delta^3 \approx \begin{pmatrix} 0.4199 \\ -0.4199 \end{pmatrix}$ (2x1)

*   **Compute $\delta^2$ (Hidden Layer 2 Error Signal):**
    *   $\frac{\partial J}{\partial \mathbf{a}^2} = (\mathbf{W}^3)^T \delta^3 = \begin{pmatrix} 0.5 & 0.9 \\ 0.6 & 0.8 \\ 0.7 & 0.7 \\ 0.8 & 0.6 \end{pmatrix} \begin{pmatrix} 0.4199 \\ -0.4199 \end{pmatrix}$
        $\approx \begin{pmatrix} 0.5(0.4199) + 0.9(-0.4199) \\ 0.6(0.4199) + 0.8(-0.4199) \\ 0.7(0.4199) + 0.7(-0.4199) \\ 0.8(0.4199) + 0.6(-0.4199) \end{pmatrix} = \begin{pmatrix} 0.4199(0.5-0.9) \\ 0.4199(0.6-0.8) \\ 0.4199(0.7-0.7) \\ 0.4199(0.8-0.6) \end{pmatrix} = \begin{pmatrix} 0.4199(-0.4) \\ 0.4199(-0.2) \\ 0.4199(0.0) \\ 0.4199(0.2) \end{pmatrix} \approx \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix}$
    *   $\text{ReLU}'(\mathbf{z}^2)$: Since $\mathbf{z}^2 = \begin{pmatrix} 0.52 \\ 1.06 \\ 1.60 \\ 0.49 \end{pmatrix}$, all elements are > 0.
        $\text{ReLU}'(\mathbf{z}^2) = \begin{pmatrix} 1 \\ 1 \\ 1 \\ 1 \end{pmatrix}$
    *   $\delta^2 = \frac{\partial J}{\partial \mathbf{a}^2} \odot \text{ReLU}'(\mathbf{z}^2) \approx \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix} \odot \begin{pmatrix} 1 \\ 1 \\ 1 \\ 1 \end{pmatrix} = \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix}$ (4x1)

*   **Compute Gradients for Layer 2 ($\mathbf{W}^2, \mathbf{b}^2$):**
    *   $\frac{\partial J}{\partial \mathbf{W}^2} = \delta^2 (\mathbf{a}^1)^T \approx \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix} \begin{pmatrix} 0.3 & 0.6 & 0.9 \end{pmatrix}$
        $\frac{\partial J}{\partial \mathbf{W}^2} \approx \begin{pmatrix} -0.1680(0.3) & -0.1680(0.6) & -0.1680(0.9) \\ -0.0840(0.3) & -0.0840(0.6) & -0.0840(0.9) \\ 0 & 0 & 0 \\ 0.0840(0.3) & 0.0840(0.6) & 0.0840(0.9) \end{pmatrix} \approx \begin{pmatrix} -0.0504 & -0.1008 & -0.1512 \\ -0.0252 & -0.0504 & -0.0756 \\ 0.0000 & 0.0000 & 0.0000 \\ 0.0252 & 0.0504 & 0.0756 \end{pmatrix}$ (4x3)
    *   $\frac{\partial J}{\partial \mathbf{b}^2} = \delta^2 \approx \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix}$ (4x1)

*   **Compute $\delta^1$ (Hidden Layer 1 Error Signal):**
    *   $\frac{\partial J}{\partial \mathbf{a}^1} = (\mathbf{W}^2)^T \delta^2 = \begin{pmatrix} 0.1 & 0.4 & 0.7 & 0.2 \\ 0.2 & 0.5 & 0.8 & 0.1 \\ 0.3 & 0.6 & 0.9 & 0.3 \end{pmatrix} \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix}$
        $\approx \begin{pmatrix} 0.1(-0.168)+0.4(-0.084)+0.7(0)+0.2(0.084) \\ 0.2(-0.168)+0.5(-0.084)+0.8(0)+0.1(0.084) \\ 0.3(-0.168)+0.6(-0.084)+0.9(0)+0.3(0.084) \end{pmatrix} = \begin{pmatrix} -0.0168 - 0.0336 + 0 + 0.0168 \\ -0.0336 - 0.0420 + 0 + 0.0084 \\ -0.0504 - 0.0504 + 0 + 0.0252 \end{pmatrix} = \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix}$
    *   $\text{ReLU}'(\mathbf{z}^1)$: Since $\mathbf{z}^1 = \begin{pmatrix} 0.3 \\ 0.6 \\ 0.9 \end{pmatrix}$, all elements are > 0.
        $\text{ReLU}'(\mathbf{z}^1) = \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix}$
    *   $\delta^1 = \frac{\partial J}{\partial \mathbf{a}^1} \odot \text{ReLU}'(\mathbf{z}^1) \approx \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix} \odot \begin{pmatrix} 1 \\ 1 \\ 1 \end{pmatrix} = \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix}$ (3x1)

*   **Compute Gradients for Layer 1 ($\mathbf{W}^1, \mathbf{b}^1$):**
    *   $\frac{\partial J}{\partial \mathbf{W}^1} = \delta^1 (\mathbf{a}^0)^T = \delta^1 \mathbf{x}^T \approx \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix} \begin{pmatrix} 1.0 & 0.5 \end{pmatrix}$
        $\frac{\partial J}{\partial \mathbf{W}^1} \approx \begin{pmatrix} -0.0336(1.0) & -0.0336(0.5) \\ -0.0672(1.0) & -0.0672(0.5) \\ -0.0756(1.0) & -0.0756(0.5) \end{pmatrix} \approx \begin{pmatrix} -0.0336 & -0.0168 \\ -0.0672 & -0.0336 \\ -0.0756 & -0.0378 \end{pmatrix}$ (3x2)
    *   $\frac{\partial J}{\partial \mathbf{b}^1} = \delta^1 \approx \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix}$ (3x1)

*   **Compute Gradient w.r.t. Input ($\mathbf{x}$):**
    *   $\frac{\partial J}{\partial \mathbf{x}} = (\mathbf{W}^1)^T \delta^1 = \begin{pmatrix} 0.1 & 0.3 & 0.5 \\ 0.2 & 0.4 & 0.6 \end{pmatrix} \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix}$
        $\approx \begin{pmatrix} 0.1(-0.0336)+0.3(-0.0672)+0.5(-0.0756) \\ 0.2(-0.0336)+0.4(-0.0672)+0.6(-0.0756) \end{pmatrix} = \begin{pmatrix} -0.00336 - 0.02016 - 0.0378 \\ -0.00672 - 0.02688 - 0.04536 \end{pmatrix} = \begin{pmatrix} -0.06132 \\ -0.07896 \end{pmatrix}$ (2x1)

**Summary of Gradients (Approximate):**
*   $\frac{\partial J}{\partial \mathbf{W}^3} \approx \begin{pmatrix} 0.2183 & 0.4451 & 0.6718 & 0.2058 \\ -0.2183 & -0.4451 & -0.6718 & -0.2058 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{b}^3} \approx \begin{pmatrix} 0.4199 \\ -0.4199 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{W}^2} \approx \begin{pmatrix} -0.0504 & -0.1008 & -0.1512 \\ -0.0252 & -0.0504 & -0.0756 \\ 0.0000 & 0.0000 & 0.0000 \\ 0.0252 & 0.0504 & 0.0756 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{b}^2} \approx \begin{pmatrix} -0.1680 \\ -0.0840 \\ 0.0000 \\ 0.0840 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{W}^1} \approx \begin{pmatrix} -0.0336 & -0.0168 \\ -0.0672 & -0.0336 \\ -0.0756 & -0.0378 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{b}^1} \approx \begin{pmatrix} -0.0336 \\ -0.0672 \\ -0.0756 \end{pmatrix}$
*   $\frac{\partial J}{\partial \mathbf{x}} \approx \begin{pmatrix} -0.0613 \\ -0.0790 \end{pmatrix}$

# Code implementation

## Imports and definitions

In [1]:
import numpy as np

# --- Activation Functions ---
def relu(z):
    """Element-wise ReLU activation."""
    return np.maximum(0, z)

def relu_derivative(z):
    """Element-wise derivative of ReLU."""
    return np.where(z > 0, 1.0, 0.0)

def softmax(z):
    """Softmax activation for a single sample or batch (along axis 1)."""
    # Shift z for numerical stability (subtract max)
    shift_z = z - np.max(z, axis=0, keepdims=True)
    exps = np.exp(shift_z)
    return exps / np.sum(exps, axis=0, keepdims=True)

### Neural net definition

In [2]:
# Use column vectors for biases and inputs
W1 = np.array([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]]) # Shape (3, 2)
b1 = np.array([[0.1], [0.1], [0.1]])              # Shape (3, 1)
W2 = np.array([[0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]) # Shape (2, 3)
b2 = np.array([[0.2], [0.2]])                     # Shape (2, 1)

# Store parameters in lists for scalability
weights = [W1, W2]
biases = [b1, b2]
num_layers = len(weights) # L = 2

### Input + sample target

In [3]:
x = np.array([[1.0], [0.5]]) # Shape (2, 1)
y = np.array([[0], [1]])     # Shape (2, 1) (one-hot)

## Forward pass

In [ ]:
# --- Forward Pass ---
activations = [x] # Store activations, a^0 = x
pre_activations = [] # Store z^l

a = x
for l in range(num_layers):
    W = weights[l]
    b = biases[l]
    z = W @ a + b
    pre_activations.append(z)
    if l < num_layers - 1: # Hidden layers use ReLU
        a = relu(z)
    else: # Output layer uses Softmax
        a = softmax(z)
    activations.append(a)

In [6]:
activations

[array([[1. ],
        [0.5]]),
 array([[0.3],
        [0.6],
        [0.9]]),
 array([[0.36818758],
        [0.63181242]])]

### Output prediction

In [7]:
y_hat = activations[-1]
print("--- Forward Pass ---")
print(f"z^1:\n{pre_activations[0]}")
print(f"a^1:\n{activations[1]}")
print(f"z^2:\n{pre_activations[1]}")
print(f"a^2 (ŷ):\n{y_hat}")

--- Forward Pass ---
z^1:
[[0.3]
 [0.6]
 [0.9]]
a^1:
[[0.3]
 [0.6]
 [0.9]]
z^2:
[[1.7 ]
 [2.24]]
a^2 (ŷ):
[[0.36818758]
 [0.63181242]]


## Backward Pass

In [9]:
gradients_W = [None] * num_layers
gradients_b = [None] * num_layers

In [10]:
# Calculate delta for the output layer (L)
# delta_L = dJ/dz^L = y_hat - y
delta = y_hat - y # delta^2 for L=2

# Gradient for the last layer
gradients_W[-1] = delta @ activations[-2].T # dJ/dW^L = delta^L * (a^(L-1))^T
gradients_b[-1] = delta                   # dJ/db^L = delta^L

# Iterate backwards from L-1 down to 1
for l in range(num_layers - 2, -1, -1):
    # Propagate delta backwards: delta^l = (W^(l+1)^T @ delta^(l+1)) .* relu'(z^l)
    W_next = weights[l+1]
    z_current = pre_activations[l]
    delta = (W_next.T @ delta) * relu_derivative(z_current)

    # Gradient for current layer l
    gradients_W[l] = delta @ activations[l].T # dJ/dW^l = delta^l * (a^(l-1))^T (activations[l] is a^l)
    gradients_b[l] = delta                  # dJ/db^l = delta^l

print("\n--- Backward Pass (Gradients) ---")
print(f"dJ/dW^2 (shape {gradients_W[1].shape}):\n{gradients_W[1]}")
print(f"dJ/db^2 (shape {gradients_b[1].shape}):\n{gradients_b[1]}")
print(f"dJ/dW^1 (shape {gradients_W[0].shape}):\n{gradients_W[0]}")
print(f"dJ/db^1 (shape {gradients_b[0].shape}):\n{gradients_b[0]}")


--- Backward Pass (Gradients) ---
dJ/dW^2 (shape (2, 3)):
[[ 0.11045627  0.22091255  0.33136882]
 [-0.11045627 -0.22091255 -0.33136882]]
dJ/db^2 (shape (2, 1)):
[[ 0.36818758]
 [-0.36818758]]
dJ/dW^1 (shape (3, 2)):
[[-0.11045627 -0.05522814]
 [-0.11045627 -0.05522814]
 [-0.11045627 -0.05522814]]
dJ/db^1 (shape (3, 1)):
[[-0.11045627]
 [-0.11045627]
 [-0.11045627]]


In [11]:
# --- Verification (Compare with manual calculation) ---
# Manual results (approx):
# dJ/dW^2 ≈ [[0.1104, 0.2208, 0.3312], [-0.1104, -0.2208, -0.3312]]
# dJ/db^2 ≈ [0.368, -0.368]^T
# dJ/dW^1 ≈ [[-0.1104, -0.0552], [-0.1104, -0.0552], [-0.1104, -0.0552]]
# dJ/db^1 ≈ [-0.1104, -0.1104, -0.1104]^T

# Check if code results match (allowing for floating point differences)
assert np.allclose(gradients_W[1], [[0.1104, 0.2208, 0.3312], [-0.1104, -0.2208, -0.3312]], atol=1e-4)
assert np.allclose(gradients_b[1], [[0.368], [-0.368]], atol=1e-4)
assert np.allclose(gradients_W[0], [[-0.1104, -0.0552], [-0.1104, -0.0552], [-0.1104, -0.0552]], atol=1e-4)
assert np.allclose(gradients_b[0], [[-0.1104], [-0.1104], [-0.1104]], atol=1e-4)

AssertionError: 